Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain, JLD
gr(size=(500,500));

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

### snippet 4.7

In [3]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [4]:
df2 = filter(row -> row[:age] >= 18, df);
first(df2, 5)

,height,weight,age,male
,Float64⍰,Float64⍰,Float64⍰,Int64⍰
1,151.765,47.8256,63.0,1
2,139.7,36.4858,63.0,0
3,136.525,31.8648,65.0,0
4,156.845,53.0419,41.0,1
5,145.415,41.2769,51.0,0


Define the Stan language model

In [5]:
weightsmodel = "
data {
 int < lower = 1 > N; // Sample size
 vector[N] height; // Predictor
 vector[N] weight; // Outcome
}

parameters {
 real alpha; // Intercept
 real beta; // Slope (regression coefficients)
 real < lower = 0 > sigma; // Error SD
}

model {
 height ~ normal(alpha + weight * beta , sigma);
}

generated quantities {
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [6]:
stanmodel = Stanmodel(name="weights", monitors = ["alpha", "beta", "sigma"],model=weightsmodel,
  output_format=:mcmcchain);
# Input data for cmdstan
heightsdata = Dict("N" => length(df2[:height]), "height" => df2[:height],
  "weight" => df2[:weight]);
# Sample using cmdstan
rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  summary=false, CmdStanDir=CMDSTAN_HOME)
# Describe the draws
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean          SD       Naive SE       MCSE      ESS
alpha 113.81208600 1.944589793 0.0307466643 0.0526215482 1000
 beta   0.90653944 0.042796305 0.0006766690 0.0011635408 1000
sigma   5.10533830 0.192599584 0.0030452668 0.0058616823 1000

Quantiles:
          2.5%        25.0%       50.0%       75.0%        97.5%   
alpha 110.11192500 112.4810000 113.778000 115.13325000 117.72615000
 beta   0.81992983   0.8778560   0.907254   0.93565475   0.98979028
sigma   4.75193700   4.9728175   5.098645   5.23056750   5.50538275



Save the chains in a JLD file

In [7]:
serialize("m4.3s.jls", chn)

chn2 = deserialize("m4.3s.jls")

describe(chn2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean          SD       Naive SE       MCSE      ESS
alpha 113.81208600 1.944589793 0.0307466643 0.0526215482 1000
 beta   0.90653944 0.042796305 0.0006766690 0.0011635408 1000
sigma   5.10533830 0.192599584 0.0030452668 0.0058616823 1000

Quantiles:
          2.5%        25.0%       50.0%       75.0%        97.5%   
alpha 110.11192500 112.4810000 113.778000 115.13325000 117.72615000
 beta   0.81992983   0.8778560   0.907254   0.93565475   0.98979028
sigma   4.75193700   4.9728175   5.098645   5.23056750   5.50538275



Should be identical to earlier result

In [8]:
describe(chn2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean          SD       Naive SE       MCSE      ESS
alpha 113.81208600 1.944589793 0.0307466643 0.0526215482 1000
 beta   0.90653944 0.042796305 0.0006766690 0.0011635408 1000
sigma   5.10533830 0.192599584 0.0030452668 0.0058616823 1000

Quantiles:
          2.5%        25.0%       50.0%       75.0%        97.5%   
alpha 110.11192500 112.4810000 113.778000 115.13325000 117.72615000
 beta   0.81992983   0.8778560   0.907254   0.93565475   0.98979028
sigma   4.75193700   4.9728175   5.098645   5.23056750   5.50538275



End of `m4.3s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*